In [169]:
import pandas as pd
from time import strptime
import locale
from datetime import date
import numpy as np

In [191]:
df_newport=pd.read_csv("https://raw.githubusercontent.com/paulineattal/Disney-Text-Mining/main/fichiers/Scrapping_Newport_Bay_Club_071222.csv",sep=';')
#et les autres ...


In [192]:
#fusionnner tous les DF 
df = df_newport 

In [193]:
df

,Names,Country,room_type,reservation_type,Duration,traveler_infos,date_review,review_title,grade_review,positive_review,negative_review,usefulness_review,hotel
0,Ganna,Ukraine,Chambre Supérieure - Côté Lac,2 nuits · Septembre 2022,Septembre 2022,Famille,Le choix des voyageurs,Good choice for visiting Disneyland,10,NaN,NaN,Utile Pas utile,Disney's Newport Bay Club
1,Diego,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 30 novembre 2022,Agréable,"6,0",la proximité du parc Disneyland Paris,"la qualité de la chambre n’était pas top, vieu...",Utile Pas utile,Disney's Newport Bay Club
2,Fabienne,Belgique,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 18 novembre 2022,déçue qualité prix,"6,0",l emplacement,la propreté,Utile Pas utile,Disney's Newport Bay Club
3,Pierre,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 14 novembre 2022,très moyen par rapport au prix,"6,0",très bon personnel dédié mais la sécurité fait...,hôtel très vieillissant et pas de shampoing ge...,Utile Pas utile,Disney's Newport Bay Club
4,Christine,France,Chambre Supérieure,1 nuit · Novembre 2022,Novembre 2022,Famille,Commentaire envoyé le 10 novembre 2022,Très bien,"8,0",La décoration de la chambre est adorable.\nOn ...,La porte ne marchait pas bien.\nQuand on met l...,Utile Pas utile,Disney's Newport Bay Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,Anonyme,Brésil,NaN,2 nuits · Janvier 2020,Janvier 2020,Famille,Commentaire envoyé le 14 janvier 2020,Exceptionnel,10,NaN,NaN,NaN,Disney's Newport Bay Club
2281,Anonyme,Royaume-Uni,NaN,2 nuits · Décembre 2019,Décembre 2019,Couple,Commentaire envoyé le 14 janvier 2020,Fabuleux,"9,2",NaN,NaN,NaN,Disney's Newport Bay Club
2282,Anonyme,Portugal,NaN,2 nuits · Janvier 2020,Janvier 2020,Couple,Commentaire envoyé le 13 janvier 2020,Fabuleux,"9,0",NaN,NaN,NaN,Disney's Newport Bay Club
2283,Anonyme,Japon,NaN,3 nuits · Janvier 2020,Janvier 2020,Famille,Commentaire envoyé le 8 janvier 2020,Décevant,"4,2",NaN,NaN,NaN,Disney's Newport Bay Club


# table date 


In [194]:
df_mois_sejour=df['reservation_type'].map(str)
for i in range(df.shape[0]):
    df_mois_sejour[i]=df_mois_sejour[i].split()[3]


df_mois_comm=df['date_review'].map(str)
for i in range(df.shape[0]):
    df_mois_comm[i]=df_mois_comm[i].split()[-2]

for i, mois in enumerate(df_mois_comm) : 
    if mois == "des" : 
        df_mois_comm[i] = df_mois_sejour[i]


In [195]:
df_annee_sejour=df['reservation_type'].map(str)
for i in range(df.shape[0]):
    df_annee_sejour[i]=df_annee_sejour[i].split()[4]

df_annee_comm=df['date_review'].map(str)
for i in range(df.shape[0]):
    df_annee_comm[i]=df_annee_comm[i].split()[-1]

for i, annee in enumerate(df_annee_comm) : 
    if annee == "voyageurs" : 
        df_annee_comm[i] = df_annee_sejour[i]


In [203]:
locale.setlocale(locale.LC_TIME,'')
list_mois_num_sejour = [strptime(mois_sejour,'%B').tm_mon for mois_sejour in df_mois_sejour]
list_mois_num_comm = [strptime(mois_comm,'%B').tm_mon for mois_comm in df_mois_comm]


d = {'mois_sejour': df_mois_sejour, 'mois_num': list_mois_num_sejour, 'annee_sejour' : df_annee_sejour}
df_date = pd.DataFrame(data=d)
df_date["date"] = pd.to_datetime(dict(year=df_date.annee_sejour, month=df_date.mois_num, day=1))


d = {'mois_comm': df_mois_comm, 'mois_num': list_mois_num_comm, 'annee_comm' : df_annee_comm}
df_date_comm = pd.DataFrame(data=d)
df_date_comm["date"] = pd.to_datetime(dict(year=df_date_comm.annee_comm, month=df_date_comm.mois_num, day=1))


delais_comm =[]

for i in range(len(df_date_comm)):
    res = abs((df_date["date"][i].year - df_date_comm["date"][i].year) * 12 + (df_date["date"][i].month - df_date_comm["date"][i].month))
    delais_comm.append(res)


#possibilité de rajouter num semaine, une vraie date au format, ou autres info ?? 


In [197]:
df_nuitee=df['reservation_type'].map(str)
for i in range(df.shape[0]):
    df_nuitee[i]=int(df_nuitee[i].split()[0])



In [198]:
d = {'Country': df["Country"], 'nuitee': df_nuitee, 'delais_comm' : delais_comm, 'traveler_infos' : df["traveler_infos"], 'review_title': df["review_title"], 'negative_review' : df["negative_review"], 'positive_review' : df["positive_review"]}
df_client = pd.DataFrame(data=d)
df_client

,Country,nuitee,delais_comm,traveler_infos,review_title,negative_review,positive_review
0,Ukraine,2,0,Famille,Good choice for visiting Disneyland,NaN,NaN
1,France,1,0,Famille,Agréable,"la qualité de la chambre n’était pas top, vieu...",la proximité du parc Disneyland Paris
2,Belgique,1,0,Famille,déçue qualité prix,la propreté,l emplacement
3,France,1,0,Famille,très moyen par rapport au prix,hôtel très vieillissant et pas de shampoing ge...,très bon personnel dédié mais la sécurité fait...
4,France,1,0,Famille,Très bien,La porte ne marchait pas bien.\nQuand on met l...,La décoration de la chambre est adorable.\nOn ...
...,...,...,...,...,...,...,...
2280,Brésil,2,0,Famille,Exceptionnel,NaN,NaN
2281,Royaume-Uni,2,1,Couple,Fabuleux,NaN,NaN
2282,Portugal,2,0,Couple,Fabuleux,NaN,NaN
2283,Japon,3,0,Famille,Décevant,NaN,NaN


In [199]:
# Liste des colonnes à transtyper en numeric
var_tofloat = ['grade_review']
for i in var_tofloat:
    df[i] = df[i].str.replace(",",".")
    df[i] = pd.to_numeric(df[i], downcast="float")

In [207]:
df_clean = pd.concat([df_client, df[['grade_review', 'room_type']]], axis = 1)
df_clean = pd.concat([df_clean, df_date], axis = 1)

In [208]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2285 entries, 0 to 2284
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Country          2283 non-null   object        
 1   nuitee           2285 non-null   object        
 2   delais_comm      2285 non-null   int64         
 3   traveler_infos   2285 non-null   object        
 4   review_title     2284 non-null   object        
 5   negative_review  498 non-null    object        
 6   positive_review  632 non-null    object        
 7   grade_review     2285 non-null   float32       
 8   room_type        2137 non-null   object        
 9   mois_sejour      2285 non-null   object        
 10  mois_num         2285 non-null   int64         
 11  annee_sejour     2285 non-null   object        
 12  date             2285 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float32(1), int64(2), object(9)
memory usage: 223.3+ KB


In [214]:
df_clean.to_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/df_clean.csv",sep=';', index=False)